<a href="https://www.kaggle.com/code/averma111/pytorch-pss3e18-vanilla?scriptVersionId=135383219" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [111]:
%%capture
!pip install ydata-profiling

In [124]:

import random
import numpy as np 
import pandas as pd 
import os
import datetime
import seaborn as sns
from tqdm.notebook import tqdm
from ydata_profiling import ProfileReport
from collections import Counter

from imblearn.over_sampling import SMOTE,SMOTEN
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from torch.nn import functional as F



import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
import itertools

In [113]:
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/playground-series-s3e18/sample_submission.csv
/kaggle/input/playground-series-s3e18/train.csv
/kaggle/input/playground-series-s3e18/test.csv


In [114]:
class Datapreparation(object):
    
    def __init__(self,root_path):
        self.root_path = root_path
        
    def get_dataframe(self,filename):
        return pd.read_csv(os.path.join(self.root_path,filename))
    
    def summary(self,text, df):
        summary = pd.DataFrame(df.dtypes, columns=['dtypes'])
        summary['null'] = df.isnull().sum()
        summary['unique'] = df.nunique()
        summary['min'] = df.min()
        summary['median'] = df.median()
        summary['max'] = df.max()
        summary['mean'] = df.mean()
        summary['std'] = df.std()
        summary['duplicate'] = df.duplicated().sum()
        return summary
    
    
    def random_split_data(self,X,y):
        return train_test_split(X, y,test_size=0.20,random_state=42)

 
    def standardization_data(self,X_data):
        scaler = StandardScaler()
        std_X_data = scaler.fit_transform(X_data)
        return std_X_data
    

    
data = Datapreparation('/kaggle/input/playground-series-s3e18')
train=data.get_dataframe('train.csv')

In [115]:
data.summary('train',train)

,dtypes,null,unique,min,median,max,mean,std,duplicate
id,int64,0,14838,0.000000,7418.500000,14837.000000,7418.500000,4283.505982,0
BertzCT,float64,0,2368,0.000000,290.987941,4069.959780,515.153604,542.456370,0
Chi1,float64,0,1259,0.000000,6.485270,69.551167,9.135189,6.819989,0
Chi1n,float64,0,3157,0.000000,4.052701,50.174588,5.854307,4.647064,0
Chi1v,float64,0,3306,0.000000,4.392859,53.431954,6.738497,5.866444,0
Chi2n,float64,0,3634,0.000000,2.970427,32.195368,4.432570,3.760516,0
Chi2v,float64,0,3725,0.000000,3.242775,34.579313,5.253221,4.925065,0
Chi3v,float64,0,3448,0.000000,1.948613,22.880836,3.418749,3.436208,0
Chi4n,float64,0,2930,0.000000,1.073261,16.072810,1.773472,1.865898,0
EState_VSA1,float64,0,719,0.000000,17.353601,363.705954,29.202823,31.728679,0


In [116]:
#profile = ProfileReport(train, title="Train Data Profiling Report")
#profile

In [117]:
y1 = train['EC1']
y2 = train['EC2']
train.drop(columns=['id','EC1','EC2','EC3','EC4','EC5','EC6'],axis=1,inplace=True)
X = train.copy()


In [129]:
X_train,X_val,y1_train,y1_val = data.random_split_data(X,y1)
print('Data splits for EC1 \n',X_train.shape,X_val.shape,y1_train.shape,y1_val.shape)

Data splits for EC1 
 (11870, 31) (2968, 31) (11870,) (2968,)


In [131]:
X_train,X_val,y2_train,y2_val = data.random_split_data(X,y2)
print('Data splits for EC2 \n',X_train.shape,X_val.shape,y2_train.shape,y2_val.shape)

Data splits for EC2 
 (11870, 31) (2968, 31) (11870,) (2968,)


In [133]:
smote = SMOTEN(random_state=42)
X_ros,y1_ros = smote.fit_resample(X_train, y1_train)
print('Original dataset shape', Counter(y1_train))
print('Resample dataset shape', Counter(y1_ros))


Original dataset shape Counter({1: 7916, 0: 3954})
Resample dataset shape Counter({1: 7916, 0: 7916})


In [134]:
X_ros,y2_ros = smote.fit_resample(X_train, y2_train)
print('Original dataset shape', Counter(y2_train))
print('Resample dataset shape', Counter(y2_ros))

Original dataset shape Counter({1: 9455, 0: 2415})
Resample dataset shape Counter({1: 9455, 0: 9455})


In [138]:
X_val_ros,y1_val_ros = smote.fit_resample(X_val, y1_val)
print('Original dataset shape', Counter(y1_val))
print('Resample dataset shape', Counter(y1_val_ros))

Original dataset shape Counter({1: 1992, 0: 976})
Resample dataset shape Counter({1: 1992, 0: 1992})


In [139]:
X_val_ros,y2_val_ros = smote.fit_resample(X_val, y2_val)
print('Original dataset shape', Counter(y2_val))
print('Resample dataset shape', Counter(y2_val_ros))

Original dataset shape Counter({1: 2400, 0: 568})
Resample dataset shape Counter({1: 2400, 0: 2400})


In [141]:
std_X_train = data.standardization_data(X_ros)
std_X_val_ros = data.standardization_data(X_val_ros)
print(std_X_train[0],std_X_val_ros[0])

[-0.44736959 -0.28433518 -0.30895399 -0.36260239 -0.36402881 -0.40836488
 -0.38300177 -0.40919831 -0.77391818  0.80020789 -0.35495059  0.09325503
  1.00946464  1.76864219  0.21925648 -0.37538094 -0.05783295 -0.11641998
  1.54337664 -0.60500365 -0.6614424  -0.47624553  0.36095092  0.58372603
 -0.50025691 -0.41020555 -0.76605138 -0.82473393 -0.75494618 -0.71874329
 -0.71928223] [-0.52293459 -0.6803927  -0.47379577 -0.48912922 -0.50298095 -0.51095892
 -0.61800484 -0.61813432 -0.19436622 -0.59422077 -0.52472259  0.07694542
  0.03515653  0.0957726   0.59797423 -0.54103524 -0.08125854 -0.45090655
  0.62962826 -0.48148737 -0.19488913 -0.25805764 -0.36719592 -0.4524227
 -0.49519869 -0.13503545 -0.33507097 -0.14858221 -0.28721812  0.79436508
  0.79469513]


In [142]:
class Tensoroperations():
    
    def __init__(self):
        super(Tensoroperations,self).__init__()
    
    def convert_to_tensor(self,X,y=None):
        X_tensor =  torch.from_numpy(X).float()   
        y_tensor = torch.from_numpy(y).float() 
        return X_tensor,y_tensor
        
    def convert_to_test_tesnor(self,X):
        X_tensor =  torch.from_numpy(X).float()
        return X_tensor
    
    def get_dataloaders(self,train_dataset,val_dataset):
        train_loaders = DataLoader(train_dataset,batch_size=32,shuffle=True)
        val_loaders = DataLoader(val_dataset,batch_size=32)
        return train_loaders,val_loaders
    
    def get_test_dataloaders(self,test_dataset,X_test):
        test_loaders = DataLoader(test_dataset,batch_size=X_test.shape[0])
        return test_loaders
        
        
    
tenops = Tensoroperations()

In [143]:
class CustomDataset(Dataset):
    
    def __init__(self,X_data,y_data=None,is_train=True):
        super().__init__()
        if is_train:
            self.X_data = X_data
            self.y_data = y_data
        else:
            self.X_data=X_train
            
    def __getitem__(self,index):
        return (self.X_data[index],self.y_data[index])
    
    def __len__(self):
        return len(self.X_data)

In [145]:
X_tensor_train,y1_tensor_train = tenops.convert_to_tensor(std_X_train,y1_ros.values)
X_tensor_val,y1_tensor_val = tenops.convert_to_tensor(std_X_val_ros,y1_val_ros.values)
print('The training tensor for EC1\n',X_tensor_train,y1_tensor_train)
print('The validation tensor for EC1\n',X_tensor_val,y1_tensor_val)

The training tensor for EC1
 tensor([[-4.4737e-01, -2.8434e-01, -3.0895e-01,  ..., -7.5495e-01,
         -7.1874e-01, -7.1928e-01],
        [-5.9814e-01, -7.6797e-01, -7.0422e-01,  ..., -8.2380e-01,
          9.0975e-01,  9.0790e-01],
        [ 2.6277e+00,  1.8630e+00,  2.1937e+00,  ...,  2.5025e+00,
         -7.1874e-01, -7.1928e-01],
        ...,
        [-4.8523e-01, -3.0963e-01, -2.6497e-01,  ...,  2.4523e-03,
         -7.1874e-01, -7.1928e-01],
        [-6.9757e-01, -7.6797e-01, -7.4097e-01,  ..., -7.3617e-01,
         -7.1874e-01, -7.1928e-01],
        [-7.3178e-01, -9.1034e-01, -5.0790e-01,  ..., -9.3647e-01,
          9.0975e-01,  9.0790e-01]]) tensor([1., 1., 1.,  ..., 0., 0., 0.])
The validation tensor for EC1
 tensor([[-5.2293e-01, -6.8039e-01, -4.7380e-01,  ..., -2.8722e-01,
          7.9437e-01,  7.9470e-01],
        [ 1.0758e-03,  5.1505e-01,  5.9708e-01,  ...,  9.8178e-02,
         -7.8189e-01, -7.8156e-01],
        [-2.0216e-01, -8.8869e-01, -9.1198e-01,  ...,  1.1081e-

In [146]:
X_tensor_train,y2_tensor_train = tenops.convert_to_tensor(std_X_train,y2_ros.values)
X_tensor_val,y2_tensor_val = tenops.convert_to_tensor(std_X_val,y2_val_ros.values)
print('The training tensor EC2\n',X_tensor_train,y2_tensor_train)
print('The validation tensor Ec2\n',X_tensor_val,y2_tensor_val)

The training tensor EC2
 tensor([[-4.4737e-01, -2.8434e-01, -3.0895e-01,  ..., -7.5495e-01,
         -7.1874e-01, -7.1928e-01],
        [-5.9814e-01, -7.6797e-01, -7.0422e-01,  ..., -8.2380e-01,
          9.0975e-01,  9.0790e-01],
        [ 2.6277e+00,  1.8630e+00,  2.1937e+00,  ...,  2.5025e+00,
         -7.1874e-01, -7.1928e-01],
        ...,
        [-4.8523e-01, -3.0963e-01, -2.6497e-01,  ...,  2.4523e-03,
         -7.1874e-01, -7.1928e-01],
        [-6.9757e-01, -7.6797e-01, -7.4097e-01,  ..., -7.3617e-01,
         -7.1874e-01, -7.1928e-01],
        [-7.3178e-01, -9.1034e-01, -5.0790e-01,  ..., -9.3647e-01,
          9.0975e-01,  9.0790e-01]]) tensor([1., 1., 0.,  ..., 0., 0., 0.])
The validation tensor Ec2
 tensor([[-5.2293e-01, -6.8039e-01, -4.7380e-01,  ..., -2.8722e-01,
          7.9437e-01,  7.9470e-01],
        [ 1.0758e-03,  5.1505e-01,  5.9708e-01,  ...,  9.8178e-02,
         -7.8189e-01, -7.8156e-01],
        [-2.0216e-01, -8.8869e-01, -9.1198e-01,  ...,  1.1081e-01,
    

In [147]:
n_input_dim = X_train.shape[1]
#Layer size
n_hidden1 = 64  # Number of hidden nodes
n_hidden2 = 64
n_output =  1   # Number of output nodes = for binary classifier

class MultiClassificationNN(torch.nn.Module):
    
    def __init__(self):
        super(MultiClassificationNN, self).__init__()
        # Number of input features is 31.
        self.layer_1 = nn.Linear(n_input_dim, n_hidden1) 
        self.layer_2 = nn.Linear(n_hidden1, n_hidden2)
        self.layer_out = nn.Linear(n_hidden2, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x
    


model_EC1 = MultiClassificationNN()
model_EC2 = MultiClassificationNN()
print(model_EC1.state_dict())
print(model_EC2.state_dict())

OrderedDict([('layer_1.weight', tensor([[ 0.0687, -0.0569, -0.1703,  ...,  0.1109, -0.0634,  0.1504],
        [-0.1124,  0.0692,  0.1070,  ...,  0.0827,  0.0904, -0.1125],
        [-0.0737, -0.0403, -0.0571,  ..., -0.0008, -0.1064,  0.1425],
        ...,
        [-0.1789,  0.0413,  0.0961,  ...,  0.1069, -0.0225,  0.0998],
        [ 0.1098,  0.1108, -0.1611,  ..., -0.1182,  0.0658,  0.0142],
        [-0.0290,  0.0386,  0.0317,  ...,  0.1327, -0.0590, -0.1075]])), ('layer_1.bias', tensor([-0.0835,  0.0815,  0.1024, -0.1016,  0.0919, -0.1423, -0.1135,  0.1713,
         0.1130,  0.1257,  0.0636,  0.0203, -0.0136,  0.1231, -0.1418, -0.0791,
        -0.1356,  0.0399,  0.1461,  0.1162,  0.1729,  0.0945,  0.1539, -0.0436,
        -0.1578,  0.0735, -0.0084, -0.0573,  0.1483, -0.0739,  0.0261, -0.0580,
         0.1733, -0.0040, -0.1220, -0.1044,  0.1280,  0.0934, -0.1523, -0.0493,
        -0.0672,  0.1547, -0.1333,  0.1285, -0.0729, -0.1584, -0.1557,  0.0350,
         0.0745,  0.1654,  0.0530, 

In [148]:
train1_dataset = CustomDataset(X_tensor_train,y1_tensor_train)
val1_dataset = CustomDataset(X_tensor_val,y1_tensor_val)

train2_dataset = CustomDataset(X_tensor_train,y2_tensor_train)
val2_dataset = CustomDataset(X_tensor_val,y2_tensor_val)

In [149]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
criterion = nn.BCEWithLogitsLoss()
     

In [150]:
dataset_EC1 = ConcatDataset([train2_dataset, val1_dataset])
dataset_EC2 = ConcatDataset([train2_dataset, val2_dataset])


num_epochs=20
batch_size=32
k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

In [151]:
class Trainer:
    
    def train_epoch(self,model,device,dataloader,loss_fn,optimizer):
        train_loss,train_correct=0.0,0
        model.train()
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = loss_fn(output,labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(output.data, 1)
            train_correct += (predictions == labels).sum().item()
        return train_loss,train_correct


    def valid_epoch(self,model,device,dataloader,loss_fn):
        valid_loss, val_correct = 0.0, 0
        model.eval()
        with torch.no_grad():
          for images, labels in dataloader:
              images,labels = images.to(device),labels.to(device)
              output = model(images)
              loss=loss_fn(output,labels.unsqueeze(1))
              valid_loss+=loss.item()*images.size(0)
              scores, predictions = torch.max(output.data,1)
              val_correct+=(predictions == labels).sum().item()

        return valid_loss,val_correct
    
trainer = Trainer()

In [152]:
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

In [153]:
class Modeltrain:
    def  call_train(self,model,dataset):
        
        for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):
            
            print('Fold {}'.format(fold + 1))
            train_sampler = SubsetRandomSampler(train_idx)
            test_sampler = SubsetRandomSampler(val_idx)
            train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
            test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)
            model.to(device)
            optimizer = optim.Adam(model.parameters(), lr=0.00001)
            
            for epoch in tqdm(range(num_epochs)):
                train_loss, train_correct=trainer.train_epoch(model,device,train_loader,criterion,optimizer)
                test_loss, test_correct=trainer.valid_epoch(model,device,test_loader,criterion)
                train_loss = train_loss / len(train_loader.sampler)
                train_acc = train_correct / len(train_loader.sampler) * 100
                test_loss = test_loss / len(test_loader.sampler)
                test_acc = test_correct / len(test_loader.sampler) * 100

                print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                             num_epochs,
                                                                                                             train_loss,
                                                                                                             test_loss,
                                                                                                             train_acc,
                                                                                                             test_acc))
                history['train_loss'].append(train_loss)
                history['test_loss'].append(test_loss)
                history['train_acc'].append(train_acc)
                history['test_acc'].append(test_acc)  
        
            foldperf['fold{}'.format(fold+1)] = history  

           
mt = Modeltrain()
torch.save(model_EC1,'MultiClassificationNN_K_Cross_EC1.pt')
torch.save(model_EC2,'MultiClassificationNN_K_Cross_EC2.pt')
mt.call_train(model_EC1,dataset_EC1)

Fold 1


  0%|          | 0/20 [00:00<?, ?it/s]

IndexError: index 4274 is out of bounds for dimension 0 with size 3984

In [ ]:
avg_train_loss_EC1 = np.mean(history['train_loss'])
avg_test_loss_EC1 = np.mean(history['test_loss'])
avg_train_acc_EC1 = np.mean(history['train_acc'])
avg_test_acc_EC1 = np.mean(history['test_acc'])

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.4f} \t Average Test Loss: {:.4f} \t Average Training Acc: {:.3f} \t Average Test Acc: {:.3f}".
      format(avg_train_loss_EC1,avg_test_loss_EC1,avg_train_acc_EC1,avg_test_acc_EC1))  


In [ ]:
mt.call_train(model_EC2,dataset_EC2)

In [ ]:
avg_train_loss_EC2 = np.mean(history['train_loss'])
avg_test_loss_EC2= np.mean(history['test_loss'])
avg_train_acc_EC2 = np.mean(history['train_acc'])
avg_test_acc_EC2 = np.mean(history['test_acc'])

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.4f} \t Average Test Loss: {:.4f} \t Average Training Acc: {:.3f} \t Average Test Acc: {:.3f}".
      format(avg_train_loss_EC2,avg_test_loss_EC2,avg_train_acc_EC2,avg_test_acc_EC2))  

In [ ]:
testl_f,tl_f,testa_f,ta_f=[],[],[],[]
k=10
for f in range(1,k):

     tl_f.append(np.mean(foldperf['fold{}'.format(f)]['train_loss']))
     testl_f.append(np.mean(foldperf['fold{}'.format(f)]['test_loss']))

     ta_f.append(np.mean(foldperf['fold{}'.format(f)]['train_acc']))
     testa_f.append(np.mean(foldperf['fold{}'.format(f)]['test_acc']))

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".format(testl_f,tl_f,testa_f,ta_f)) 

In [ ]:
diz_ep = {'train_loss_ep':[],'test_loss_ep':[],'train_acc_ep':[],'test_acc_ep':[]}

for i in range(num_epochs):
      diz_ep['train_loss_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['train_loss'][i] for f in range(k)]))
      diz_ep['test_loss_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['test_loss'][i] for f in range(k)]))
      diz_ep['train_acc_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['train_acc'][i] for f in range(k)]))
      diz_ep['test_acc_ep'].append(np.mean([foldperf['fold{}'.format(f+1)]['test_acc'][i] for f in range(k)]))

In [ ]:
# Plot losses
plt.figure(figsize=(10,8))
plt.semilogy(diz_ep['train_loss_ep'], label='Train')
plt.semilogy(diz_ep['val_loss_ep'], label='Vallidation')
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.grid()
plt.legend()
plt.title('EC1 Loss')
plt.show()
     

In [ ]:

# Plot accuracies
plt.figure(figsize=(10,8))
plt.semilogy(diz_ep['train_acc_ep'], label='Train')
plt.semilogy(diz_ep['val_acc_ep'], label='Validation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.grid()
plt.legend()
plt.title('EC1 Accuracy')
plt.show()